In [40]:
import json
import random
import requests
import time
from datetime import datetime, timedelta

In [41]:
response = requests.get(f"{ORION_URL}/version")
print(response.json())

{'orionld version': '1.4.0', 'orion version': '1.15.0-next', 'uptime': '0 d, 3 h, 55 m, 14 s', 'git_hash': 'nogitversion', 'compile_time': 'Sat Aug 26 06:19:09 UTC 2023', 'compiled_by': 'root', 'compiled_in': '', 'release_date': 'Sat Aug 26 06:19:09 UTC 2023', 'doc': 'https://fiware-orion.readthedocs.org/en/master/'}


In [42]:
def generate_single_charging_session_entity(session_number):
    """
    Generate a single charging session test entity
    """
    base_time = datetime(2025, 1, 28, 8, 0, 0)
    session_statuses = ["ended", "active", "suspended", "preparing"]
    charging_point_prefixes = [
        "ParisSaclay30-1-P-",
        "ParisSaclay30-2-P-", 
        "ParisSaclay30-3-P-",
        "ParisBoulogne-1-P-",
        "ParisBoulogne-2-P-"
    ]
    
    # Generate a unique ID with a timestamp to avoid duplication 生成包含时间戳的唯一ID，避免重复
    timestamp = int(time.time() * 1000)  # Millisecond timestamp 毫秒时间戳
    session_id = f"perf_test_{session_number:03d}_{timestamp}"
    charging_point_prefix = random.choice(charging_point_prefixes)
    charging_point_id = f"{charging_point_prefix}{random.randint(1, 20):02d}"
    status = random.choice(session_statuses)
    
    # Generate random times
    start_time_offset = timedelta(
        days=random.randint(0, 30),
        hours=random.randint(0, 23),
        minutes=random.randint(0, 59)
    )
    start_time = base_time + start_time_offset
    
    session_duration = timedelta(
        hours=random.randint(0, 8),
        minutes=random.randint(30, 59)
    )
    end_time = start_time + session_duration
    
    entity = {
        "@context": "https://smartdatamodels.org/context.jsonld",
        "id": f"urn:ngsi-ld:ChargingSessionTest:session-{session_id}",
        "type": "ChargingSessionTest",
        
        "refChargingPoint": {
            "type": "Relationship",
            "object": f"urn:ngsi-ld:ChargingPoint:{charging_point_id}"
        },
        
        "sessionId": {
            "type": "Property",
            "value": session_id
        },
        
        "sessionStatus": {
            "type": "Property",
            "value": status
        },
        
        "sessionStartTime": {
            "type": "Property",
            "value": {
                "@type": "DateTime",
                "@value": start_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
            },
            "observedAt": start_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        },
        
        "sessionEndTime": {
            "type": "Property",
            "value": {
                "@type": "DateTime",
                "@value": end_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
            },
            "observedAt": end_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        }
    }
    
    return entity

def test_single_entity_creation(entity, orion_url="http://localhost:1026"):
    """
    Test creating a single entity with detailed debug output
    """
    headers = {
        "Content-Type": "application/ld+json",
        "Accept": "application/json"
    }
    
    print("🔍 DEBUG - Entity to be sent:")
    print(json.dumps(entity, indent=2))
    print("\n" + "="*50)
    
    start_time = time.time()
    
    try:
        response = requests.post(
            f"{orion_url}/ngsi-ld/v1/entities",
            headers=headers,
            data=json.dumps(entity),
            timeout=30
        )
        
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        
        print(f"⏱️  Request took: {elapsed_ms:.1f}ms")
        print(f"📡 Response status: {response.status_code}")
        print(f"📄 Response headers: {dict(response.headers)}")
        print(f"💬 Response text: {response.text}")
        
        success = response.status_code == 201
        return success, elapsed_ms, response.status_code, response.text
        
    except Exception as e:
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        print(f"❌ Exception occurred: {e}")
        return False, elapsed_ms, "ERROR", str(e)

def create_single_entity_with_timing(entity, orion_url="http://localhost:1026"):
    """
    Create a single entity in Orion and measure the time
    """
    headers = {
        "Content-Type": "application/ld+json",
        "Accept": "application/json"
    }
    
    start_time = time.time()
    
    try:
        response = requests.post(
            f"{orion_url}/ngsi-ld/v1/entities",
            headers=headers,
            data=json.dumps(entity),
            timeout=30
        )
        
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        
        success = response.status_code == 201
        
        # 处理409错误（实体已存在）
        if response.status_code == 409:
            return False, elapsed_ms, response.status_code, "Entity already exists"
        
        return success, elapsed_ms, response.status_code, response.text
        
    except Exception as e:
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        return False, elapsed_ms, "ERROR", str(e)

def run_performance_test(iterations=20, orion_url="http://localhost:1026"):
    """
    Run performance test by creating entities one by one
    """
    results = []
    
    print(f"🚀 Starting performance test: {iterations} iterations")
    print("=" * 60)
    
    for i in range(1, iterations + 1):
        print(f"Test {i:2d}/{iterations}: ", end="")
        
        # Generate entity
        entity = generate_single_charging_session_entity(i)
        session_id = entity["sessionId"]["value"]
        
        # Create entity with timing
        success, elapsed_ms, response_code, error_message = create_single_entity_with_timing(entity, orion_url)
        
        # Record result
        result = {
            "iteration": i,
            "session_id": session_id,
            "success": success,
            "elapsed_ms": elapsed_ms,
            "response_code": response_code,
            "error_message": error_message,
            "timestamp": datetime.now().isoformat()
        }
        results.append(result)
        
        # Print immediate result
        status_icon = "✅" if success else "❌"
        print(f"{status_icon} {session_id} - {elapsed_ms:.1f}ms - {response_code}")
        
        # Small delay between requests
        time.sleep(0.1)
    
    return results

def analyze_performance_results(results):
    """
    Analyze and display performance statistics
    """
    successful_results = [r for r in results if r["success"]]
    failed_results = [r for r in results if not r["success"]]
    
    print("\n" + "=" * 60)
    print("📊 PERFORMANCE ANALYSIS")
    print("=" * 60)
    
    print(f"Total requests:        {len(results)}")
    print(f"Successful requests:   {len(successful_results)} ({len(successful_results)/len(results)*100:.1f}%)")
    print(f"Failed requests:       {len(failed_results)} ({len(failed_results)/len(results)*100:.1f}%)")
    
    if successful_results:
        response_times = [r["elapsed_ms"] for r in successful_results]
        
        min_time = min(response_times)
        max_time = max(response_times)
        avg_time = sum(response_times) / len(response_times)
        
        sorted_times = sorted(response_times)
        p50 = sorted_times[len(sorted_times) // 2]
        p95_index = int(len(sorted_times) * 0.95)
        p95 = sorted_times[min(p95_index, len(sorted_times) - 1)]
        
        print(f"\n⏱️  Response Time Statistics (ms):")
        print(f"Minimum:              {min_time:.1f} ms")
        print(f"Maximum:              {max_time:.1f} ms")
        print(f"Average:              {avg_time:.1f} ms")
        print(f"Median (P50):         {p50:.1f} ms")
        print(f"95th Percentile:      {p95:.1f} ms")
    
    if failed_results:
        print(f"\n❌ First 3 Failed Requests:")
        for result in failed_results[:3]:
            print(f"   {result['iteration']:2d}. {result['session_id']} - {result['response_code']}")
            if result['error_message']:
                print(f"       Error: {result['error_message'][:200]}...")

# delet_test_entities
def clear_test_entities(orion_url="http://localhost:1026"):
    """
    清理所有测试实体
    """
    try:
        # 查询所有ChargingSessionTest类型的实体
        response = requests.get(
            f"{orion_url}/ngsi-ld/v1/entities",
            params={"type": "ChargingSessionTest", "limit": 1000},
            headers={"Accept": "application/json"}
        )
        
        if response.status_code == 200:
            entities = response.json()
            print(f"Found {len(entities)} entities to delete")
            
            deleted_count = 0
            for entity in entities:
                entity_id = entity["id"]
                delete_response = requests.delete(f"{orion_url}/ngsi-ld/v1/entities/{entity_id}")
                if delete_response.status_code == 204:
                    deleted_count += 1
                    print(f"✅ Deleted: {entity_id}")
                else:
                    print(f"❌ Failed to delete: {entity_id}")
            
            print(f"Successfully deleted {deleted_count} entities")
            return deleted_count
        else:
            print(f"Failed to query entities: {response.status_code}")
            return 0
            
    except Exception as e:
        print(f"Error clearing entities: {e}")
        return 0

# =============================================================================
# 使用示例
# =============================================================================

print("🔧 Orion Context Broker Performance Test")
print("适用于 Jupyter Notebook")
print()
print("现在按顺序运行以下代码:")
print()
print("# 1. 检查Orion连接")
print("check_orion_connection()")
print()
print("# 2. 清理之前的测试数据（可选）")
print("clear_test_entities()")
print()
print("# 3. 生成测试实体")
print("test_entity = generate_single_charging_session_entity(1)")
print("print('Generated entity:')")
print("print(json.dumps(test_entity, indent=2))")
print()
print("# 4. 测试单个实体创建（带详细调试信息）")
print("success, elapsed_ms, response_code, error_message = test_single_entity_creation(test_entity)")
print()
print("# 5. 如果第4步成功，运行完整测试")
print("results = run_performance_test(iterations=5)  # 先试5次")
print()
print("# 6. 分析结果")
print("analyze_performance_results(results)")
print()
print("=" * 60)

🔧 Orion Context Broker Performance Test
适用于 Jupyter Notebook

现在按顺序运行以下代码:

# 1. 检查Orion连接
check_orion_connection()

# 2. 清理之前的测试数据（可选）
clear_test_entities()

# 3. 生成测试实体
test_entity = generate_single_charging_session_entity(1)
print('Generated entity:')
print(json.dumps(test_entity, indent=2))

# 4. 测试单个实体创建（带详细调试信息）
success, elapsed_ms, response_code, error_message = test_single_entity_creation(test_entity)

# 5. 如果第4步成功，运行完整测试
results = run_performance_test(iterations=5)  # 先试5次

# 6. 分析结果
analyze_performance_results(results)



In [48]:
clear_test_entities()

Found 10 entities to delete
✅ Deleted: urn:ngsi-ld:ChargingSessionTest:session-perf_test_001_1751047283973
✅ Deleted: urn:ngsi-ld:ChargingSessionTest:session-perf_test_002_1751047284082
✅ Deleted: urn:ngsi-ld:ChargingSessionTest:session-perf_test_003_1751047284196
✅ Deleted: urn:ngsi-ld:ChargingSessionTest:session-perf_test_004_1751047284309
✅ Deleted: urn:ngsi-ld:ChargingSessionTest:session-perf_test_005_1751047284422
✅ Deleted: urn:ngsi-ld:ChargingSessionTest:session-perf_test_006_1751047284535
✅ Deleted: urn:ngsi-ld:ChargingSessionTest:session-perf_test_007_1751047284652
✅ Deleted: urn:ngsi-ld:ChargingSessionTest:session-perf_test_008_1751047284766
✅ Deleted: urn:ngsi-ld:ChargingSessionTest:session-perf_test_009_1751047284879
✅ Deleted: urn:ngsi-ld:ChargingSessionTest:session-perf_test_010_1751047285008
Successfully deleted 10 entities


10

In [44]:
test_entity = generate_single_charging_session_entity(1)
print('Generated entity:')
print(json.dumps(test_entity, indent=2))

Generated entity:
{
  "@context": "https://smartdatamodels.org/context.jsonld",
  "id": "urn:ngsi-ld:ChargingSessionTest:session-perf_test_001_1751047283949",
  "type": "ChargingSessionTest",
  "refChargingPoint": {
    "type": "Relationship",
    "object": "urn:ngsi-ld:ChargingPoint:ParisBoulogne-2-P-14"
  },
  "sessionId": {
    "type": "Property",
    "value": "perf_test_001_1751047283949"
  },
  "sessionStatus": {
    "type": "Property",
    "value": "preparing"
  },
  "sessionStartTime": {
    "type": "Property",
    "value": {
      "@type": "DateTime",
      "@value": "2025-02-25T12:43:00.000Z"
    },
    "observedAt": "2025-02-25T12:43:00.000Z"
  },
  "sessionEndTime": {
    "type": "Property",
    "value": {
      "@type": "DateTime",
      "@value": "2025-02-25T20:39:00.000Z"
    },
    "observedAt": "2025-02-25T20:39:00.000Z"
  }
}


In [45]:
#success, elapsed_ms, response_code, error_message = test_single_entity_creation(test_entity)


In [ ]:
results = run_performance_test(iterations=10)  

🚀 Starting performance test: 10 iterations
Test  1/10: ✅ perf_test_001_1751047283973 - 9.0ms - 201


Test  2/10: ✅ perf_test_002_1751047284082 - 13.7ms - 201
Test  3/10: ✅ perf_test_003_1751047284196 - 12.1ms - 201
Test  4/10: ✅ perf_test_004_1751047284309 - 12.5ms - 201
Test  5/10: ✅ perf_test_005_1751047284422 - 12.9ms - 201
Test  6/10: ✅ perf_test_006_1751047284535 - 13.9ms - 201
Test  7/10: ✅ perf_test_007_1751047284652 - 13.2ms - 201
Test  8/10: ✅ perf_test_008_1751047284766 - 13.2ms - 201
Test  9/10: ✅ perf_test_009_1751047284879 - 27.5ms - 201
Test 10/10: ✅ perf_test_010_1751047285008 - 262.9ms - 201


In [47]:
analyze_performance_results(results)


📊 PERFORMANCE ANALYSIS
Total requests:        10
Successful requests:   10 (100.0%)
Failed requests:       0 (0.0%)

⏱️  Response Time Statistics (ms):
Minimum:              9.0 ms
Maximum:              262.9 ms
Average:              39.1 ms
Median (P50):         13.2 ms
95th Percentile:      262.9 ms
